Loading required libraries. 
* folium - used for data visualization on an interactive map
* pandas - used to simplify data manipulation
* time - used for timeouts to do not overload geocoding API (I'm ok to wait for now, yes)
* geopy - used for geocoding, the process of getting latitude and longitude from the adress 

In [1]:
import folium
import pandas
import time
from geopy.geocoders import ArcGIS
from geopy.exc import GeocoderTimedOut

Load the source data file. 

All of the information is publicy available and provided by [WEB PORTAL OF OPEN DATA KHARKOV (Веб-портал відкритих даних Харкова)](https://data.city.kharkov.ua/):  
* [Information about advertising media (Інформація про рекламні засоби)](https://data.city.kharkov.ua/passport/2UpP-kle4-gr76-3CZd.html)

In [2]:
adsFileInput = 'data/adTools_05102021.csv'

Geocoding - get lantitude and longitude for all of the ads adresses.  
Remove unneeded columns.

In [ ]:
%%time

def get_geocode(address, number, attempt=1, max_attempts=5):
    addr = address.replace(' - ', ' & ') + ', Харків, Україна'
    print(f'#{number}', end='\r', flush=True)
    try:
        time.sleep(1)
        return gis.geocode(addr)
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            time.sleep(5)
            return get_geocode(addr, number, attempt=attempt+1)
        raise
        
gis = ArcGIS()
df = pandas.read_csv(adsFileInput, header=[1])

df.drop('Електронна адреса розповсюджувача реклами', 1, inplace=True)
df.drop('Телефон розповсюджувача реклами', 1, inplace=True)

df['Координати'] = [get_geocode(a,n) for a,n in zip(df['Адреса місця розташування'], df['№ з/п.'])]

df

Map renderring: 
1. Group all entries with the same adress to single popups 
2. Define markers color based on the ads contract expriration date
3. Put markers on the map according to ads location

In [ ]:
kh = gis.geocode("Харків, Україна")
ads_map = folium.Map([kh.latitude, kh.longitude], zoom_start=14)
uniq_df = df.drop_duplicates('Адреса місця розташування', ignore_index=True)

green_fg = folium.FeatureGroup(name="Термін договору дійсний")
red_fg = folium.FeatureGroup(name="Термін договору завершився")
orange_fg = folium.FeatureGroup(name="Декілька договорів")

def get_popup(rows):
    rows = rows.rename(columns={
        'Повне найменування заявника':'Заявник', 
        'Адреса місця розташування': 'Адреса розташування',
        'Дата рішення про надання дозволу': 'Дата надання дозволу'
    })
    
    return folium.Popup(
        rows.loc[:, 'Заявник':'Дата надання дозволу'].to_html(
            classes="table table-striped table-hover table-condensed table-responsive",
            index=False
        )
    )

def get_icon(rows):
    colors = []
    for index, row in rows.iterrows():
        if pandas.to_datetime(rows.iloc[0]['Строк дії дозволу'], format='%Y-%m-%d') < pandas.to_datetime('now'):
            colors.append(['red', 'hourglass-end'])
        else:
            colors.append(['green', 'hourglass-start'])
    if len(set([i[0] for i in colors])) > 2:
        color, icon = ['orange', 'hourglass-half']
    else:
        color, icon = colors[0]
                           
    return folium.Icon(
        color=color,
        icon=icon,
        prefix='fa'
    )

def put_marker(addr):
    rows = df.loc[df['Адреса місця розташування'] == addr]
    marker = folium.Marker(
        location = [rows.iloc[0]['Координати'].latitude, rows.iloc[0]['Координати'].longitude],
        popup = get_popup(rows),
        icon = get_icon(rows)
    )
    if marker.icon.options['markerColor'] == 'green':
        fg = green_fg.add_child(marker)
    elif marker.icon.options['markerColor'] == 'red':
        fg = red_fg.add_child(marker)
    else:
        fg = orange_fg.add_child(marker)
    ads_map.add_child(fg)

                           
uniq_df['Адреса місця розташування'].apply(lambda addr: put_marker(addr))

folium.LayerControl().add_to(ads_map)

ads_map

Save the resulted map as HTML file.

In [ ]:
ads_map.save('index.html')